<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-07-27 13:52:10--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11848 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-07-27 13:52:10 (102 MB/s) - ‘utils.py’ saved [11848/11848]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.7 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [10]:
model._name = 'combined-model-v2'

In [11]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy', 
                                              verbose=1,
                                              patience=6,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name)
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_accuracy',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [13]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
533/533 [==============================] - ETA: 0s - loss: 13.1129 - auc: 0.8167 - accuracy: 0.7356

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 4266s 8s/step - loss: 13.1129 - auc: 0.8167 - accuracy: 0.7356 - val_loss: 0.6526 - val_auc: 0.8788 - val_accuracy: 0.7926
Epoch 2/15
533/533 [==============================] - ETA: 0s - loss: 7.8116 - auc: 0.9162 - accuracy: 0.8432

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


...

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 499s 933ms/step - loss: 0.5494 - auc: 0.9974 - accuracy: 0.9874 - val_loss: 0.8909 - val_auc: 0.8640 - val_accuracy: 0.8372
Epoch 13/15
533/533 [==============================] - ETA: 0s - loss: 0.3835 - auc: 0.9986 - accuracy: 0.9904

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2/assets


533/533 [==============================] - 512s 958ms/step - loss: 0.3835 - auc: 0.9986 - accuracy: 0.9904 - val_loss: 0.9874 - val_auc: 0.8540 - val_accuracy: 0.8392
Epoch 14/15
533/533 [==============================] - 477s 892ms/step - loss: 0.3404 - auc: 0.9987 - accuracy: 0.9925 - val_loss: 1.0337 - val_auc: 0.8493 - val_accuracy: 0.8375
Epoch 15/15
533/533 [==============================] - 461s 863ms/step - loss: 0.2995 - auc: 0.9990 - accuracy: 0.9940 - val_loss: 1.1199 - val_auc: 0.8439 - val_accuracy: 0.8323


In [14]:
!cp  -r /content/logs/checkpoints/combined-model-v2 /content/drive/MyDrive/checkpoints/combined-v2-aug